## Generate Quizzes of Course Materials (PDF) using LlamaIndex, Astra DB (Apache Cassandra), and Gradient's open-source models, including LLama2/Mixtral.

[**Link to my GitHub**](https://github.com/baltsat)

Click on the link below to open a Colab version of the notebook. You will be able to create your own version.

<a href="https://colab.research.google.com/gist/Baltsat/a7821ff379ce13ece8ac894faefd8428/ai-quiz-generation.ipynb" target="_blank"><img height="40" alt="Run your own notebook in Colab" src = "https://colab.research.google.com/assets/colab-badge.svg"></a>

# Installation

In [ ]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

!pip install -q langchain
!pip install -q sentence-transformers

!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.4/270.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 r

# Import OS & JSON Modules

In [ ]:
import os
import json
from google.colab import userdata

# os.environ['GRADIENT_ACCESS_TOKEN'] = 'PUT_YOURS_HERE'
# os.environ['GRADIENT_WORKSPACE_ID'] =  'PUT_YOURS_HERE'

# Import Cassandra & llama Index

In [ ]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings import GradientEmbedding, LangchainEmbedding
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.llms import GradientBaseModelLLM, HuggingFaceLLM
from llama_index.vector_stores import CassandraVectorStore

In [ ]:
import cassandra
print (cassandra.__version__)

3.29.0


In [ ]:
from pprint import pprint as pprint

# Connect to the VectorDB

In [ ]:
# This secure connect bundle is autogenerated when you donwload your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-temp-db.zip'
}

# This token json file is autogenerated when you download your token,
# if yours is different update the file name below
with open("konstantin.baltsat@edu.esiee.fr-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

ERROR:cassandra.datastax.cloud:HTTP Error 401: Unauthorized
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/cassandra/datastax/cloud/__init__.py", line 137, in read_metadata_info
    response = urlopen(url, context=config.ssl_context, timeout=timeout)
  File "/usr/lib/python3.10/urllib/request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/lib/python3.10/urllib/request.py", line 525, in open
    response = meth(req, response)
  File "/usr/lib/python3.10/urllib/request.py", line 634, in http_response
    response = self.parent.error(
  File "/usr/lib/python3.10/urllib/request.py", line 563, in error
    return self._call_chain(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 643, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 401: Unauthorized


DriverException: Unable to connect to the metadata service at https://3c4fc467-6c52-444c-8464-0ebddaaed953-us-east1.db.astra.datastax.com:29080/metadata. Check the cluster status in the cloud console. 

# Define the Gradient's Model Adapter for LLAMA-2

In [ ]:
# llm = GradientBaseModelLLM(
#     base_model_slug="llama2-70b-chat",
#     max_tokens=510,
# )

llm = GradientBaseModelLLM(
    base_model_slug="mixtral-8x7b-instruct",
    max_tokens=510,
)

# Configure Gradient embeddings

In [ ]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

# embed_model = LangchainEmbedding(HuggingFaceEmbeddings())

In [ ]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=512,
)

set_global_service_context(service_context)

# Load the PDFs

In [ ]:
documents = SimpleDirectoryReader("/content/documents").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 5 document(s).


# Setup and Query Index

In [ ]:
index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()

In [ ]:
structure = '''{
  "questions": [
    {
      "question": "Чему равна площадь прямоугольного треугольника со сторонами 20 см и 30 см",
      "options": [
        {
          "option": "600 см^2",
          "correct": true,
          "tip": "Так как треугольник прямоугольный, воспользуемся формулой из главы 'Площади': S = a * b, где a, b - стороны искомого треугольника."
        },
        ...
      ]
    },
    ...
  ]
}
'''

structure = ''

prompt = "Составь тест: 2 вопроса с 4 вариантами ответа на каждый по материалу урока из загруженных файлов. Из 4 вариантов должен быть только один верный. К каждому варианту ответа должен быть предоставлено пояснение (tip) - цитатата из текста, если вариант верный, и пояснение или опровергающая цитата, если вариант неверный. Пришли тест мне в строго формате без лишних символов и объяснений сохраняя структуру json. Не вставляй в текст ссылки, графические знаки или что-либо еще, пришли только символьный текст в нужном формате. Все специальные символы запрещены. Пример ожидаемого ответа: " + structure

prompt = 'Сформулируй тест: вопрос по теме урока из загруженного файла и возможный ответ к нему. Пришли только вопрос в формате строки. Формат твоего вывода: <Вопрос> <Короткий ответ> <Пояснение с цитатой из материала, объясняющее почему ответ верен>. Пример: <Чему равна площадь прямоугольного треугольника со сторонами 20 см и 30 см?><600 см^2><Так как треугольник прямоугольный, воспользуемся формулой из главы "Площади": S = a * b, где a, b - стороны искомого треугольника.>' + structure
pprint(prompt)

('Сформулируй тест: вопрос по теме урока из загруженного файла и возможный '
 'ответ к нему. Пришли только вопрос в формате строки. Формат твоего вывода: '
 '<Вопрос> <Короткий ответ> <Пояснение с цитатой из материала, объясняющее '
 'почему ответ верен>. Пример: <Чему равна площадь прямоугольного треугольника '
 'со сторонами 20 см и 30 см?><600 см^2><Так как треугольник прямоугольный, '
 'воспользуемся формулой из главы "Площади": S = a * b, где a, b - стороны '
 'искомого треугольника.>')


In [ ]:
response = query_engine.query(prompt).response
response

'<What is the value of R1 to maximize the frequency range for which the circuit looks inductive, given R2 = 50ME and the condition Rj // R1 = 50kΩ?> <R1 = 50kΩ> <According to equation (8), to maximize the frequency range for which the circuit looks inductive, we should minimize R1. Given the condition Rj // R1 = 50kΩ, we can find R1 by using the formula for parallel resistance: R1 = (Rj * R_total) / (Rj + R_total) = (50kΩ * 50ME) / (50kΩ + 50ME) = 50kΩ.>'

In [ ]:
import re
import json

def decode_to_json(text):
    # Pattern to match content within "<>"
    pattern = r'<(.*?)>'
    matches = re.findall(pattern, text)

    # Initialize the structure of the JSON
    result = {"questions": []}
    question = None

    for match in matches:
        # Check if the current match is a question
        if match.endswith('?'):
            question = {"question": match, "options": []}
            result["questions"].append(question)
        else:
            # Check if the current match contains an option (e.g., "R1 = 2.6MΩ")
            option_match = re.search(r'R1\s*=\s*([\d.]+[MKΩ]+)', match)
            if option_match:
                # Found an R1 option, create a new option entry
                option_text = "R1 = " + option_match.group(1)
                option = {"option": option_text, "correct": True, "tip": match}
                if question:
                    question["options"].append(option)
            elif question and question["options"]:
                # If there's already an option, append the current match as additional information to the last option's tip
                question["options"][-1]['tip'] += " " + match

    return json.dumps(result, indent=2)

decoded_json = decode_to_json(response)
print(decoded_json)
data = json.loads(decoded_json)
data['questions'][0]['question']


{
  "questions": [
    {
      "question": "What is the value of R1 to maximize the frequency range for which the circuit looks inductive, given R2 = 50ME and the condition Rj // R1 = 50k\u03a9?",
      "options": []
    }
  ]
}


'What is the value of R1 to maximize the frequency range for which the circuit looks inductive, given R2 = 50ME and the condition Rj // R1 = 50kΩ?'

In [ ]:
prompt = 'You are given the following json structure of some educational test: ' + decoded_json
prompt += '\n Continue it in such a way to add 3 more irrelevant and incorrect answers. In "tip" field give an explanation why it is not correct based on files donwloaded. Send me the answer in the same well-compiled json format. '

#shorter prompt
prompt = "Create 2 (two) invalid distraction answers for this question: "+ data['questions'][0]['question']
prompt += '\nФормат твоего вывода: 1. <Короткий неверный ответ> <Пояснение с цитатой из материала, объясняющее, почему ответ неверен>.'
print(prompt)

Create 2 (two) invalid distraction answers for this question: What is the value of R1 to maximize the frequency range for which the circuit looks inductive, given R2 = 50ME and the condition Rj // R1 = 50kΩ?
Формат твоего вывода: 1. <Короткий неверный ответ> <Пояснение с цитатой из материала, объясняющее, почему ответ неверен>.


In [ ]:
response = query_engine.query(prompt).response
print(response)

1. R1 = 0 Ω, < A value of R1 equal to 0 Ohms would mean that R1 is a short circuit, which is not possible in this case. The condition Rj // R1 = 50kΩ would not be satisfied, and the circuit would not function properly. The patient protection requirement mentioned in the new context also makes this answer invalid.>
2. R1 = 1 GΩ, < The goal is to maximize the frequency range for which the circuit looks inductive, but increasing the value of R1 beyond a certain limit will not help achieve this goal. In fact, it may have the opposite effect. According to the problem, the optimal solution is R1 = 50kΩ. The new context about patient protection does not affect the validity of this answer, as a value of 1 GΩ for R1 is still not the optimal solution.>


In [ ]:
import re
import json

def add_invalid_answers(llm_answers, existing_data):
    # Define a pattern to match the LLM answers structure
    pattern = r'(\d+)\)\s+(R1\s*=\s*[\d.]+\s*[GMk]?Ω).*?"(.*?)"\s+\(page_label:\s+(\d+)\)'
    matches = re.findall(pattern, llm_answers, re.DOTALL)
    print(matches)

    for match in matches:
        # Extracting the components of each invalid answer
        _, option, explanation, page_label = match

        # Construct the option dictionary
        invalid_option = {
            "option": option,
            "correct": False,  # Marking the answer as incorrect
            "tip": f'{explanation} (page_label: {page_label})'
        }

        # Assuming there's only one question and we're adding invalid options to it
        if existing_data['questions']:
            existing_data['questions'][0]['options'].append(invalid_option)

    return existing_data

# Example usage
llm_answers = response

existing_data = {
  "questions": [
    {
      "question": "What is the value of R1 in the circuit with the given component values and conditions, and how does it affect the frequency range of the circuit?",
      "options": [
        {
          "option": "R1 = 2.6MΩ",
          "correct": True,
          "tip": "R1 = 2.6MΩ. To maximize the frequency range for which the circuit looks inductive, R1 should be minimized. Solution 1 yields a smaller value of R1, which results in a predicted time constant of 100 seconds. According to equation (11), RjR2=2.6X1012, and from the first solution of the equations, R1 = RjR2 / R2 = 2.6MΩ for Rj = 1GE (10^9 Ω), R2 = 100MΩ (10^7 Ω), and C1 = 1pF."
        }
      ]
    }
  ]
}

# Transform and add invalid answers to the existing data
updated_data = add_invalid_answers(llm_answers, existing_data)

# Print the updated JSON structure
print(json.dumps(updated_data, indent=2))


[]
{
  "questions": [
    {
      "question": "What is the value of R1 in the circuit with the given component values and conditions, and how does it affect the frequency range of the circuit?",
      "options": [
        {
          "option": "R1 = 2.6M\u03a9",
          "correct": true,
          "tip": "R1 = 2.6M\u03a9. To maximize the frequency range for which the circuit looks inductive, R1 should be minimized. Solution 1 yields a smaller value of R1, which results in a predicted time constant of 100 seconds. According to equation (11), RjR2=2.6X1012, and from the first solution of the equations, R1 = RjR2 / R2 = 2.6M\u03a9 for Rj = 1GE (10^9 \u03a9), R2 = 100M\u03a9 (10^7 \u03a9), and C1 = 1pF."
        }
      ]
    }
  ]
}


In [ ]:
!python --version

Python 3.10.12
